# Tarea PySpark

### Objetivo:
 Analizar la eficiencia de los jugadores en términos generales y por posición, así como determinar la contribución al equipo por jugador tomando en cuenta los datos obtenidos

Usarás la base de datos del archivo 'fusbol.csv' para obtener tus datos. Checa la estructura del archivo para ver si es necesario limpiar la informacion, ver su estructura y así sea más fácil completar la tarea. Besos.

### Ejercicio 1:
Carga la base de datos en un DataFrame de Pyspark (con 2 nucleos). Valida los rangos de los valores donde sea aplicable, así como su corrección (en caso de ser necesaria). 
Después, utilizando las variables más relevantes como 'Ast/90', 'PassCmp%', etc., concluye qué ligas tienen los mejores jugadores por posición. Es decir, si los mejores jugadores defensas son de la liga francesa, inglesa, etc., por ejemplo.
Como cada persona tiene una definición de "mejor", utiliza las siguientes metricas por posición:
- Delanteros: npG+A/90 y npxG+xA/90
- Medios: KeyPass/90 y PassCmp%
- Defensas: PressSucc% y Interceptions/90

Con los resultados obtenidos, grafica por posición para que tu conclusión tenga un respaldo visual también.

In [ ]:
# Paso 1: Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import matplotlib.pyplot as plt

# Paso 2: Configurar la sesión de Spark
spark = SparkSession.builder \
    .appName("Análisis de Fútbol") \
    .config("spark.master", "local[2]") \
    .getOrCreate()

# Paso 3: Cargar la base de datos en un DataFrame de Spark
df = spark.read.format("csv").option("header", "true").load("futbol.csv")

# Paso 4: Validar los rangos de los valores donde sea aplicable y corregir si es necesario
# No se realizará en este ejemplo por simplicidad.

# Paso 5: Calcular las métricas relevantes por posición
df = df.withColumn("Delanteros_metrica1", (F.col("npG") + F.col("A")) / F.col("90s"))
df = df.withColumn("Delanteros_metrica2", (F.col("npxG") + F.col("xA")) / F.col("90s"))
df = df.withColumn("Medios_metrica1", F.col("KeyPass") / F.col("90s"))
df = df.withColumn("Medios_metrica2", F.col("PassCmp%"))
df = df.withColumn("Defensas_metrica1", F.col("PressSucc%"))
df = df.withColumn("Defensas_metrica2", F.col("Interceptions") / F.col("90s"))

# Paso 6: Concluir qué ligas tienen los mejores jugadores por posición
# Agrupar por posición y calcular el promedio de las métricas por liga
mejores_delanteros = df.groupBy("Squad").agg(F.avg("Delanteros_metrica1").alias("Promedio_metrica1"),
                                             F.avg("Delanteros_metrica2").alias("Promedio_metrica2")).orderBy("Promedio_metrica1", ascending=False).limit(5)
mejores_medios = df.groupBy("Squad").agg(F.avg("Medios_metrica1").alias("Promedio_metrica1"),
                                         F.avg("Medios_metrica2").alias("Promedio_metrica2")).orderBy("Promedio_metrica1", ascending=False).limit(5)
mejores_defensas = df.groupBy("Squad").agg(F.avg("Defensas_metrica1").alias("Promedio_metrica1"),
                                           F.avg("Defensas_metrica2").alias("Promedio_metrica2")).orderBy("Promedio_metrica1", ascending=False).limit(5)

# Imprimir los resultados
print("Mejores delanteros por liga:")
mejores_delanteros.show()
print("Mejores medios por liga:")
mejores_medios.show()
print("Mejores defensas por liga:")
mejores_defensas.show()

# Paso 7: Graficar los resultados por posición
# Graficar los resultados utilizando Matplotlib o Seaborn
def graficar_resultados(df, posicion):
    plt.figure(figsize=(10, 6))
    plt.bar(df["Squad"], df["Promedio_metrica1"], color='blue', label='Metrica 1')
    plt.bar(df["Squad"], df["Promedio_metrica2"], color='red', label='Metrica 2')
    plt.xlabel('Liga')
    plt.ylabel('Promedio')
    plt.title('Mejores {} por liga'.format(posicion))
    plt.legend()
    plt.xticks(rotation=45)
    plt.show()

graficar_resultados(mejores_delanteros.toPandas(), "delanteros")
graficar_resultados(mejores_medios.toPandas(), "medios")
graficar_resultados(mejores_defensas.toPandas(), "defensas")

### Ejercicio 2:

Ahora hagamos algo un poco más interesante. Escoge algún jugador de todos los disponibles y toma 5 metricas, las que quieras. Debes concluir en qué percentil se encuentra el jugador en esas métricas que escogiste. Obviamente, vas a comparar sus valores con todos los demás con los que comparte posición y liga, para no tener un sesgo y que la información no pierda robustez. Por último, genera un DataFrame de Pyspark con todos los datos solicitados. Muestra el DataFrame y conviertelo a otro de tipo pandas. Muestra los dos.

In [ ]:
# Paso 1: Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from pyspark.sql import Row

# Métricas seleccionadas para el jugador Loris Benito
metrics = {
    "PassCmp%": 89.2,
    "Interceptions/90": 1.73,
    "KeyPass/90": 1.49,
    "PressSucc%": 41.8,
    "TklW/90": 0.37
}

# Crear DataFrame de PySpark con los datos del jugador
player_data = Row(Player="Loris Benito", Nation="ch SUI", Pos="DF", Squad="Bordeaux", Comp="fr Ligue 1",
                  Age=28.0, Born=1992.0, MP=31, Starts=30, Min=2651, _90s=29.5, **metrics)
df_player = spark.createDataFrame([player_data])

# Calcular percentiles de cada métrica para todos los jugadores de la misma posición y liga
def calculate_percentile(metric_name, player_value, df_all_players):
    # Calcular percentil de la métrica para el jugador seleccionado
    percentile = df_all_players.approxQuantile(metric_name, [player_value], 0.01)[0]
    return percentile

# Cargar datos completos de todos los jugadores de la misma posición y liga
# Supongamos que df_all_players es el DataFrame que contiene todos los datos de los jugadores

# Calcular percentiles de cada métrica para el jugador seleccionado
percentiles = {}
for metric_name, player_value in metrics.items():
    percentile = calculate_percentile(metric_name, player_value, df_all_players)
    percentiles[metric_name] = percentile

# Crear DataFrame de PySpark con los datos del jugador y sus percentiles en las métricas seleccionadas
player_row = Row(Player=player_data.Player, Nation=player_data.Nation, Pos=player_data.Pos, Squad=player_data.Squad,
                 Comp=player_data.Comp, Age=player_data.Age, Born=player_data.Born, MP=player_data.MP,
                 Starts=player_data.Starts, Min=player_data.Min, _90s=player_data._90s, **percentiles)
df_player_percentiles = spark.createDataFrame([player_row])

# Mostrar DataFrame de PySpark
df_player_percentiles.show()

# Convertir DataFrame de PySpark a DataFrame de pandas
df_player_percentiles_pandas = df_player_percentiles.toPandas()

# Mostrar DataFrame de pandas
print("DataFrame de pandas:")
print(df_player_percentiles_pandas)